In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [71]:
train=pd.read_csv('f:/ml4trainsparse.csv',usecols=[1,2,3])
test=pd.read_csv('f:/ml4testsparse.csv',usecols=[1,2,3])

In [72]:
no_movies = 3953
no_users = 6041

In [73]:
import time

In [74]:
rated= [set() for i in range(no_movies) ]
dictionary={}

In [75]:
deviation=np.zeros(no_movies*no_movies).reshape(no_movies,no_movies)
cardinality=np.zeros(no_movies*no_movies).reshape(no_movies,no_movies)

In [76]:
for i, row in enumerate(train.itertuples(), 1):
    usr=int(row.userId)
    mov=int(row.movieId)
    rate=row.rating
    
    rated[mov].add(usr)
    dictionary[(usr,mov)]=rate

In [77]:
for i in range(no_movies):
    t1=time.time()
    
    for j in range(i+1,no_movies):
        tmprate = list(rated[i].intersection(rated[j]))
        
        uoutput=0
        c=0
        for uid in tmprate:
            uoutput=uoutput+(dictionary[(uid,j)]-dictionary[(uid,i)])
            c=c+1
            
        deviation[i][j]=uoutput
        deviation[j][i]=-uoutput
        cardinality[i][j]=c
        cardinality[j][i]=c

In [78]:
test.describe()

,userId,movieId,rating
count,10865.000000,10865.000000,10865.000000
mean,3985.905108,1956.120110,3.412241
std,1845.833670,1139.847254,1.199479
min,1.000000,1.000000,1.000000
25%,2481.000000,1022.000000,3.000000
50%,4508.000000,1966.000000,4.000000
75%,5682.000000,2916.000000,4.000000
max,6040.000000,3952.000000,5.000000


In [79]:
number=test['userId'].count()

In [80]:
import math

In [81]:
rmse_normal2=0
mae_normal2=0
err_gim=0
err_gpim=0
c_gim=0
c_gpim=0
tp=0
fp=0
tn=0
fn=0
for i, row in enumerate(test.itertuples(), 1):
    usr=int(row.userId)
    mov=int(row.movieId)
    actual=row.rating
    pred_normal2=0
    pred_weighted2=0
    sum_weighted2=0
    list_of_all_movies_rate_by_the_current_user=train[train['userId']==usr]
    length=list_of_all_movies_rate_by_the_current_user.shape[0]
    cnt=0
    for j,r2 in enumerate(list_of_all_movies_rate_by_the_current_user.itertuples(),1):
        curmov=r2.movieId
        currate=r2.rating
        if cardinality[curmov][mov]!=0:
            pred_normal2+=(currate+(deviation[curmov][mov]/ cardinality[curmov][mov]))
            cnt+=1
    if cnt!=0:
        predicted_normal2=(pred_normal2/cnt)
    else:
        predicted_normal2=0
    if predicted_normal2>5:
        predicted_normal2=5
    if predicted_normal2 <0:
        predicted_normal2=0
    
    if actual >=3:
        err_gim+=math.fabs(actual-predicted_normal2)
        c_gim+=1
    if predicted_normal2>=3:
        err_gpim+=math.fabs(actual-predicted_normal2)
        c_gpim+=1
    if actual>=3 and predicted_normal2>=3:
        tp+=1
    elif actual<3 and predicted_normal2>=3:
        fp+=1
    elif actual>=3 and predicted_normal2<3:
        fn+=1
    else:
        tn+=1
    rmse_normal2+=((actual-predicted_normal2)**2)
    mae_normal2+=abs(actual-predicted_normal2)
    

In [82]:
print(mae_normal2/number)

1.67115717648


In [83]:
import math

In [84]:
print(math.sqrt(rmse_normal2/number))

2.2424399637794536


In [85]:
err_gim/c_gim

1.7785889666506391

In [86]:
err_gpim/c_gpim

0.862302471180833

In [87]:
tp/(tp+fp)

0.8661207389068749

In [88]:
tp/(tp+fn)

0.5338654771686818